In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import matplotlib.ticker as plticker
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from collections import defaultdict
from typing import Optional
from scipy.integrate import solve_ivp
from scipy.fftpack import diff as psdiff
import torch
import pytorch_lightning as pl
from torch import nn
from collections import defaultdict
import numpy as np

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympde'))

from data.dataset import PDEDataset, PDEDataModule
from data.utils import d_to_LT
from misc.utils import load_obj

from model.setup import setup_model
from model.learner import Learner


from viz.plot_pde_data import plot_1ds, plot_1d_dict
from misc.utils import load_obj, save_obj

from run import parse_options, main

MLP
CNN
ResNet
FNO1d


In [9]:
nets = {net: np.nan for net in ['MLP', 'CNN', 'ResNet', 'FNO1d']}


for net, param in nets.items():
    args = parse_options(notebook=True)
    args.train = True
    args.pde_name = 'KdV'
    args.net = net # 'MLP'
    args.max_epochs = 10
    args.num_workers = 0
    args.persistent_workers = False

    args.do_return = True
    args.data_dir = '../data/v1'

    model, trainer, datamodule = main(args)

    param = sum(p.numel() for p in model.parameters() if p.requires_grad)
    nets[net] = param





Seed set to 42
/home/eliasd/.conda/envs/sympde/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/eliasd/.conda/envs/sympde/lib/python3.10/site- ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42




###	Version:  None 	###
###	Name:  datav1_netMLP_KdV_aug0_seed42 	###


Selecting 500 out of the 500 train samples!
Selecting 100 out of the 100 val samples!
Selecting 100 out of the 100 test samples!


###	Version:  None 	###
###	Name:  datav1_netCNN_KdV_aug0_seed42 	###




GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42


Selecting 500 out of the 500 train samples!
Selecting 100 out of the 100 val samples!
Selecting 100 out of the 100 test samples!


###	Version:  None 	###
###	Name:  datav1_netResNet_KdV_aug0_seed42 	###




GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42


Selecting 500 out of the 500 train samples!
Selecting 100 out of the 100 val samples!
Selecting 100 out of the 100 test samples!


###	Version:  None 	###
###	Name:  datav1_netFNO1d_KdV_aug0_seed42 	###




GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Selecting 500 out of the 500 train samples!
Selecting 100 out of the 100 val samples!
Selecting 100 out of the 100 test samples!


In [10]:
nets

{'MLP': 32105, 'CNN': 1002885, 'ResNet': 3158533, 'FNO1d': 10851589}

In [14]:
us, dxs, dts = batch = next(iter(datamodule.train_dataloader()))

In [15]:
out = model(batch)
out[0].shape, out[1].shape

(torch.Size([10, 5, 256]), torch.Size([10, 5, 256]))

In [ ]:
print([b.shape for b in batch])

In [ ]:
batch2 = (us[:, :12, :200], dxs, dts)
print([b.shape for b in batch2])

In [ ]:
x = batch2[0]

fc0 = nn.Linear(12, 256)
print('fc0', fc0)
x = x.permute(0, 2, 1)
print("x.shape: ", x.shape)
x = fc0(x)
print("x.shape: ", x.shape)

In [ ]:
model(batch2)

In [ ]:
us.shape

In [4]:
# import torch
# import torchvision

# class myMLP(torch.nn.Module):
#     def __init__(self, 
#             time_history: int,
#             time_future: int,
#             hidden_channels: list = [256, 256, 256],
#             activation = torch.nn.ReLU,
#         ):
#         super(myMLP, self).__init__()

#         channels = hidden_channels + [time_future]
#         self.mlp = torchvision.ops.MLP(
#             in_channels = time_history, 
#             hidden_channels = channels,
#             norm_layer = None,
#             activation_layer = activation,
#         )

#     def forward(self, x):
#         x = x.permute(0, 2, 1)
#         x = self.mlp(x)
#         print(x.shape)
#         x = x.permute(0, 2, 1)
#         print(x.shape)
#         return x

mlpmodel = myMLP(time_history = 40, time_future = 10, hidden_channels = [100, 100, 100])

out = mlpmodel(us[:, :, :])
out.shape

torch.Size([10, 10, 256])

In [ ]:
# print pytorch version
print(torch.__version__)
assert False

### E2CNN

In [ ]:
datamodule = PDEDataModule(
    pde_name = args.pde_name, 
    data_dir = args.data_dir, 
    batch_size = args.batch_size, 
    num_workers = args.num_workers,
    n_splits = [int(n_split) for n_split in args.n_splits],
    generators = args.generators,
    persistent_workers = args.persistent_workers,
)
datamodule.setup()
batch = next(iter(datamodule.train_dataloader()))

The 1D ResNet has 8 convolutional layers using residual connections and increasing receptive fields (kernel size 3 to kernel size 15) a

In [ ]:
net = CNN(
    # block = BasicBlock1d,
    # num_blocks = [2, 2, 2, 2],
    time_history = 10,
    time_future  = 5,
)

model = Learner(net, nn.MSELoss())

y_pred, y = model(batch)
y_pred = y_pred.detach().numpy()

In [ ]:
plt.imshow(y_pred[0], aspect='auto')

In [ ]:
# args.generators = False
# args.do_return = False

# # model, trainer, datamodule = main(args)

In [ ]:
dataset_train = datamodule.train_dataloader().dataset

In [ ]:
datamodule.setup(stage='test')
batch = next(iter(datamodule.train_dataloader()))

In [ ]:
# set to inference mode
model.eval()

In [ ]:
def flip_batch(batch):
    batch_flipped = batch.copy()
    batch_flipped[0] = batch_flipped[0].flip(2)
    return batch_flipped

def flip_y(y):
    y_flipped = y.clone()
    y_flipped = y_flipped.flip(2)
    return y_flipped

def pred(batch):
    y_pred, y = model(batch)
    y_pred = y_pred.detach()
    y = y.detach()
    return y_pred, y

In [ ]:
y_pred1, y1 = pred(batch)
y_pred_flipped1, y_flipped1 = flip_y(y_pred1), flip_y(y1)

In [ ]:
batch_flipped = flip_batch(batch)
y_pred_flipped2, y_flipped2 = pred(batch_flipped)

In [ ]:
(y_flipped1 == y_flipped2).all(), (y_pred_flipped1 == y_pred_flipped2).all()

In [ ]:
_, dx, dt = batch

In [ ]:
diff = (y_pred_flipped1-y_pred_flipped2)

In [ ]:
plots = [y_pred_flipped1, y_pred_flipped2, diff]

plot_1ds(torch.cat(plots).unsqueeze(0), dx, dt, vminmax = False)

In [ ]:
plots_dict = {
    'y_flips' : [y_pred_flipped1, y_pred_flipped2, y_pred_flipped1-y_pred_flipped2]
}

plot_1d_dict(plots_dict)

In [ ]:
model.log_fig(batch_flipped, y_pred_flipped1)

In [ ]:
model.log_fig(batch_flipped, y_pred_flipped2)


## G-FNO

In [ ]:
num_channels = 1
modes = 12
width = 10 # for G-FNO p4, for G-FNO p4m then 7
T_in = initial_step = 10 # input time (history)
reflection = False # False if p4, if p4m then True
grid_type = 'symmetric'
strategy = 'teacher_forcing'
T = 20
noise_std = 0

In [ ]:
from model.networks.gfno import GFNO2d, pde_data

In [ ]:
model = GFNO2d(
    num_channels=num_channels,
    modes=modes,
    width=width,
    initial_step=initial_step,
    reflection=reflection,
    grid_type=grid_type,
)

In [ ]:
# x = us[:, :10, :]
# model(x)

In [ ]:
data_path = '../ext_repos/G-FNO/data'

train_path = os.path.join(data_path, 'ns_V0.01_N10_T30_cos4_super.mat')
data = scipy.io.loadmat(os.path.expandvars(train_path))['u'].astype(np.float32)
data = torch.from_numpy(data).float()
print(data.shape)
data = data.unsqueeze(-1)
print(data.shape)

In [ ]:
train_dataset = pde_data(data, strategy=strategy, T_in=T_in, T_out=T, std=noise_std)

In [ ]:
x, y = train_dataset[0]

In [ ]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)

In [ ]:
xx, yy = next(iter(dataloader))

In [ ]:
x_pred = model(xx)

In [ ]:
x_pred.shape

In [ ]:
x.shape, y.shape